In [9]:
import os
import warnings
import keras
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import newaxis
from keras.layers.core import  Activation, Dropout
from keras.models import Model
from keras.layers import Input, Embedding,LSTM, Dense,TimeDistributed
from keras import metrics
from sklearn.metrics import mean_squared_error
from math import sqrt
from numpy import concatenate
#from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
#from keras.models import Sequential


In [2]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(),list()
    for i in range(n_in, 0 ,-1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1,i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = concat(cols, axis =1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace = True)
    return agg

In [15]:
dataset_stock = read_csv('fai1.csv', header = 0, index_col=0)
values_stock = dataset_stock.values
encoder = LabelEncoder()
values_stock[:,0] = encoder.fit_transform(values_stock[:,0])
values_stock = values_stock.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_stock = scaler.fit_transform(values_stock)
reframed_stock = series_to_supervised(scaled_stock, 1, 1)
#reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis = 1, inplace=True)
print(reframed_stock.head())

In [41]:
dataset_gap = read_csv('gap1.csv', header = 0, index_col=0)
values_gap = dataset_gap.values
#encoder = LabelEncoder()
#values_gap[:,0] = encoder.fit_transform(values_gap[:,0])
values_gap = values_gap.astype('float32')
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled_gap = scaler.fit_transform(values_gap)
#reframed_gap = series_to_supervised(scaled_gap, 1, 1)
#reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis = 1, inplace=True)
#print(reframed_gap.head())
print values_gap

[[ 402.]
 [ 391.]
 [ 429.]
 [ 384.]
 [ 417.]
 [ 383.]
 [ 410.]
 [ 387.]
 [ 384.]
 [ 383.]
 [ 407.]
 [ 407.]
 [ 370.]
 [ 413.]
 [ 447.]
 [ 451.]
 [ 403.]
 [ 396.]
 [ 409.]
 [ 380.]
 [ 398.]
 [ 397.]
 [ 423.]
 [ 417.]
 [ 382.]
 [ 393.]
 [ 423.]
 [ 433.]
 [ 390.]
 [ 399.]
 [ 398.]
 [ 389.]
 [ 371.]
 [ 389.]
 [ 409.]
 [ 364.]
 [ 389.]
 [ 403.]
 [ 375.]
 [ 361.]
 [ 405.]
 [ 401.]
 [ 407.]
 [ 372.]
 [ 423.]
 [ 418.]
 [ 394.]
 [ 374.]
 [ 362.]
 [ 402.]
 [ 397.]
 [ 383.]
 [ 386.]
 [ 396.]
 [ 417.]
 [ 397.]
 [ 400.]
 [ 381.]
 [ 387.]
 [ 417.]
 [ 443.]
 [ 391.]
 [ 412.]
 [ 379.]
 [ 416.]
 [ 384.]
 [ 400.]
 [ 411.]
 [ 442.]
 [ 417.]
 [ 406.]
 [ 389.]
 [ 417.]
 [ 420.]
 [ 384.]
 [ 423.]
 [ 396.]
 [ 375.]
 [ 363.]
 [ 358.]
 [ 407.]
 [ 406.]
 [ 409.]
 [ 419.]
 [ 396.]
 [ 382.]
 [ 406.]
 [ 398.]
 [ 342.]
 [ 408.]
 [ 384.]
 [ 428.]
 [ 395.]
 [ 409.]
 [ 402.]
 [ 396.]
 [ 372.]
 [ 454.]
 [ 419.]
 [ 401.]
 [ 396.]
 [ 383.]
 [ 395.]
 [ 392.]
 [ 414.]
 [ 411.]
 [ 385.]
 [ 415.]
 [ 440.]
 [ 407.]
 [ 429.]
 

In [42]:
dataset_fai = read_csv('fai1.csv', header = 0, index_col=0)
values_fai = dataset_fai.values
#encoder = LabelEncoder()
#values_fai[:,0] = encoder.fit_transform(values_fai[:,0])
values_fai = values_fai.astype('float32')
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled_fai = scaler.fit_transform(values_fai)
#reframed_fai = series_to_supervised(scaled_fai, 1, 1)
#reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis = 1, inplace=True)
#print(reframed_fai.head())
print values_fai

[[ 13.10000038]
 [ 13.19999981]
 [ 13.10000038]
 [ 13.19999981]
 [ 13.10000038]
 [ 11.69999981]
 [ 11.60000038]
 [ 11.69999981]
 [ 11.60000038]
 [ 11.60000038]
 [ 15.69999981]
 [ 15.69999981]
 [ 15.69999981]
 [ 15.69999981]
 [ 15.69999981]
 [ 19.29999924]
 [ 19.29999924]
 [ 19.39999962]
 [ 19.29999924]
 [ 19.39999962]
 [ 15.60000038]
 [ 15.60000038]
 [ 15.60000038]
 [ 15.60000038]
 [ 15.69999981]
 [ 14.        ]
 [ 14.        ]
 [ 14.        ]
 [ 14.10000038]
 [ 14.10000038]
 [  8.69999981]
 [  8.69999981]
 [  8.69999981]
 [  8.60000038]
 [  8.60000038]
 [ 16.79999924]
 [ 16.79999924]
 [ 16.79999924]
 [ 16.79999924]
 [ 16.79999924]
 [ 12.89999962]
 [ 12.89999962]
 [ 12.89999962]
 [ 12.89999962]
 [ 12.89999962]
 [ 12.89999962]
 [ 12.89999962]
 [ 12.89999962]
 [ 12.89999962]
 [ 16.39999962]
 [ 16.39999962]
 [ 16.39999962]
 [ 16.39999962]
 [ 16.39999962]
 [ 14.10000038]
 [ 14.19999981]
 [ 14.10000038]
 [ 14.10000038]
 [ 14.10000038]
 [ 15.        ]
 [ 15.        ]
 [ 15.        ]
 [ 15.  

In [43]:
values_stock = reframed_stock.values
n_train_hours = 673
train_stock = values_stock[:n_train_hours, :]

test_stock = values_stock[n_train_hours:, :]

train_X_stock, train_y_stock = train_stock[:,:-1], train_stock[:, -1]

test_stock_X, test_stock_y = test_stock[:,:-1],test_stock[:,-1]

train_X_stock = train_X_stock.reshape((train_X_stock.shape[0], 1, train_X_stock.shape[1]))
test_stock_X = test_stock_X.reshape((test_stock_X.shape[0], 1, test_stock_X.shape[1]))
print(train_X_stock.shape, train_y_stock.shape, test_stock_X.shape, test_stock_y.shape)
print(test_stock_X, test_stock_y)

((673, 1, 1), (673,), (5, 1, 1), (5,))
(array([[[ 0.56146789]],

       [[ 0.50825691]],

       [[ 0.51009172]],

       [[ 0.50091743]],

       [[ 0.48440367]]], dtype=float32), array([ 0.50825691,  0.51009172,  0.50091743,  0.48440367,  0.49541286], dtype=float32))


In [65]:
stock_input = Input(shape=(train_X_stock.shape[1], train_X_stock.shape[2]), dtype='float32', name='stock_input')
print stock_input

Tensor("stock_input_2:0", shape=(?, 1, 1), dtype=float32)


In [45]:
lstm_out_stock = LSTM(32)(stock_input)
lstm_out_stock = Dense(1)(lstm_out_stock)
print lstm_out_stock

Tensor("dense_7/BiasAdd:0", shape=(?, 1), dtype=float32)


In [47]:
stock_output = Dense(1, activation='linear', name='stock_output')(lstm_out_stock)

In [61]:
#values_gap = reframed_gap.values
#values_gap = values_gap.values
n_train_hours = 673
train_gap = values_gap[:n_train_hours, :]

test_gap = values_gap[n_train_hours:, :]

train_X_gap, train_y_gap = train_gap[:,:-1], train_gap[:, -1]

test_gap_X, test_gap_y = test_gap[:,:-1],test_gap[:,-1]

train_X_gap = train_X_gap.reshape((train_X_gap.shape[0], 1, train_X_gap.shape[1]))
test_gap_X = test_gap_X.reshape((test_gap_X.shape[0], 1, test_gap_X.shape[1]))
print(train_X_gap.shape, train_y_gap.shape, test_gap_X.shape, test_gap_y.shape)
print(test_gap_X, test_gap_y)

((673, 1, 0), (673,), (6, 1, 0), (6,))
(array([], shape=(6, 1, 0), dtype=float32), array([ 342.,  402.,  385.,  374.,  418.,  407.], dtype=float32))


In [64]:
gap_input = Input(shape=(train_X_gap.shape[0], train_X_gap.shape[1]),dtype='float32', name='gap_input')
#gap_input = Input(shape=(train_X_gap.shape[1],),dtype='float32', name='gap_input')
#gap_input = TimeDistributed(Dense(1, activation='linear'))(gap_input)
print gap_input
#lstm_out_stock = TimeDistributed(Dense(1, activation='linear'))(lstm_out_stock)
print lstm_out_stock

Tensor("gap_input_7:0", shape=(?, 673, 1), dtype=float32)
Tensor("dense_7/BiasAdd:0", shape=(?, 1), dtype=float32)


In [49]:
#lstm_out_stock_embedding = Embedding(output_dim=512, input_dim = 10000, input_length=1)(lstm_out_stock)

#print lstm_out_stock_embedding

In [50]:
#merg_first = keras.layers.concatenate([lstm_out_stock_embedding, gap_input])
merg_first = keras.layers.concatenate([lstm_out_stock, gap_input])
merg_first = Embedding(output_dim=512, input_dim = 10000, input_length=1)(merg_first)


print merg_first
lstm_out_gap = LSTM(64)(merg_first)
lstm_out_gap = LSTM(64)(merg_first)
lstm_out_gap = Dense(1)(lstm_out_gap)
print lstm_out_gap                    

Tensor("embedding_5/Gather:0", shape=(?, 2, 512), dtype=float32)
Tensor("dense_8/BiasAdd:0", shape=(?, 1), dtype=float32)


In [51]:
gap_output = Dense(1, activation='linear', name='gap_output')(lstm_out_gap)

In [52]:
#values_fai = reframed_fai.values
n_train_hours = 673
train_fai = values_fai[:n_train_hours, :]

test_fai = values_fai[n_train_hours:, :]

train_X_fai, train_y_fai = train_fai[:,:-1], train_fai[:, -1]

test_fai_X, test_fai_y = test_fai[:,:-1],test_fai[:,-1]

train_X_fai = train_X_fai.reshape((train_X_fai.shape[0], 1, train_X_fai.shape[1]))
test_fai_X = test_fai_X.reshape((test_fai_X.shape[0], 1, test_fai_X.shape[1]))
print(train_X_fai.shape, train_y_fai.shape, test_fai_X.shape, test_fai_y.shape)
print(test_fai_X, test_fai_y)

((673, 1, 0), (673,), (6, 1, 0), (6,))
(array([], shape=(6, 1, 0), dtype=float32), array([ 20.70000076,  20.70000076,  20.70000076,  20.70000076,
        20.70000076,  20.70000076], dtype=float32))


In [53]:
#gap_input = Input(shape=(train_X_gap.shape[1],),dtype='float32', name='gap_input')
#gap_input = TimeDistributed(Dense(1, activation='linear'))(gap_input)

#lstm_out_stock = TimeDistributed(Dense(1, activation='linear'))(lstm_out_stock)

fai_input = Input(shape=(train_X_fai.shape[1], ),dtype='float32', name='fai_input')
#gap_input = TimeDistributed(Dense(1, activation='linear'))(gap_input)
print fai_input
print lstm_out_gap

Tensor("fai_input_2:0", shape=(?, 1), dtype=float32)
Tensor("dense_8/BiasAdd:0", shape=(?, 1), dtype=float32)


In [54]:
#lstm_out_gap_embedding = Embedding(output_dim=512, input_dim = 10000, input_length=1)(lstm_out_gap)

#print lstm_out_gap_embedding

In [55]:
merg_second = keras.layers.concatenate([lstm_out_gap, fai_input])
merg_second = Embedding(output_dim=512, input_dim = 10000, input_length=1)(merg_second)

print merg_second
lstm_out_fai = LSTM(128)(merg_second)
lstm_out_fai = Dense(1)(lstm_out_fai)
#merg_second = keras.layers.concatenate([lstm_out_gap_embedding, fai_input])
print lstm_out_fai

Tensor("embedding_6/Gather:0", shape=(?, 2, 512), dtype=float32)
Tensor("dense_9/BiasAdd:0", shape=(?, 1), dtype=float32)


In [56]:
x = Dense(64, activation='linear')(lstm_out_fai)

main_output = Dense(1, activation='linear', name='main_output')(x)

In [57]:
model = Model(inputs=[stock_input, gap_input, fai_input], outputs=[stock_output, gap_output, main_output])

In [58]:
model.compile(optimizer='rmsprop',
              loss={'stock_output':'mse', 'gap_output':'mse' ,'main_output':'mse'},
             loss_weights={'main_output': 1., 'stock_output': 0.2, 'gap_output': 0.2})

In [59]:
validation_data=({'stock_input':test_stock_X, 
               'gap_input': test_gap_X, 
               'fai_input': test_fai_X},
              {'stock_output':test_stock_y , 
               'gap_output': test_gap_y ,
               'main_output':test_fai_y})
history=model.fit({'stock_input':train_X_stock, 
           'gap_input': train_X_gap, 
           'fai_input': train_X_fai},
         {'stock_output':train_y_stock , 
          'gap_output': train_y_gap ,
          'main_output':train_y_fai},
         epochs=50, batch_size=1, validation_data=validation_data, verbose=0, shuffle=False)

ValueError: Error when checking input: expected gap_input to have 2 dimensions, but got array with shape (673, 1, 0)